# Q Risk Report for HTT

This notebook generates a comprehensive quantitative risk report using EWMA covariance estimation.

In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Configuration & Inputs

In [43]:
# ============================================================================
# CONFIGURATION - Edit these parameters as needed
# ============================================================================

product = "clbr"

# Bucket definitions
front = ["A01", "A02", "A03", "A04"]
mid   = ["A05", "A06", "A07", "A08"]
back  = ["A09", "A10", "A11", "A12", "A13", "A14", "A15"]

# EWMA decay parameters (lambda)
lambda_front = 0.97
lambda_mid   = 0.98
lambda_back  = 0.99
lambda_cross = 0.985  # For cross-bucket correlations (if using Option 2)

# EWMA initialization
ewma_init_obs = 60  # Use first N observations for sample covariance initialization

# Data file path
data_file = "data_.csv"

# Delta summary file path
delta_summary_file = "delta_summary.csv"

# Holidays file path
holidays_file = "holidays.csv"

# Load delta summary
delta_summary_df = pd.read_csv(delta_summary_file)

# Load holidays and create set for fast lookup
holidays_df = pd.read_csv(holidays_file, header=None, names=['date'])
holidays_df = holidays_df.dropna()  # Remove empty rows
holidays_df['date'] = pd.to_datetime(holidays_df['date'], format='%m/%d/%Y', errors='coerce')
holidays_df = holidays_df.dropna()  # Remove any rows that couldn't be parsed
holiday_dates = set(holidays_df['date'].dt.date)  # Use .date() for date-only comparison
print(f"  Loaded {len(holiday_dates)} holiday dates from holidays.csv")

# Build complete contract-to-node mapping for all tenors in delta_summary.csv
# Map tenors chronologically to A01-A15 (first 15 tenors in the CSV)
# The order in delta_summary.csv is: H6, J6, K6, M6, N6, Q6, U6, V6, X6, Z6, F7, G7, H7, J7, K7, ...
unique_tenors = delta_summary_df['Tenor'].unique()
contract_to_node = {}
for idx, tenor in enumerate(unique_tenors[:15]):  # Only map first 15 to A01-A15
    node_code = f"A{idx+1:02d}"
    contract_to_node[tenor] = node_code

# Find product column (case-insensitive matching)
product_upper = product.upper()
available_products = [col for col in delta_summary_df.columns if col.upper() == product_upper]
if not available_products:
    # Try to find a close match
    all_products = [col for col in delta_summary_df.columns if col != 'Tenor']
    raise ValueError(f"Product '{product}' not found in delta_summary.csv. Available products: {all_products}")
product_column = available_products[0]

# Build pos_codes dictionary from delta_summary.csv for the selected product
pos_codes = {}
for _, row in delta_summary_df.iterrows():
    tenor = row['Tenor']
    position = row[product_column]
    # Only include non-zero positions
    if abs(position) > 1e-10:  # Use small threshold to handle floating point precision
        pos_codes[tenor] = float(position)

print("Configuration loaded:")
print(f"  Product: {product} (from column: {product_column})")
print(f"  Front bucket: {front}")
print(f"  Mid bucket: {mid}")
print(f"  Back bucket: {back}")
print(f"  Lambda (front/mid/back): {lambda_front}/{lambda_mid}/{lambda_back}")
print(f"  Loaded {len(pos_codes)} positions from delta_summary.csv")
print(f"  Contract-to-node mappings: {len(contract_to_node)} contracts")

Configuration loaded:
  Product: clbr (from column: CLBR)
  Front bucket: ['A01', 'A02', 'A03', 'A04']
  Mid bucket: ['A05', 'A06', 'A07', 'A08']
  Back bucket: ['A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15']
  Lambda (front/mid/back): 0.97/0.98/0.99
  Loaded 6 positions from delta_summary.csv
  Contract-to-node mappings: 15 contracts


## 2. Load & Clean Data

In [44]:
# Load data
df_raw = pd.read_csv(data_file)

# Parse date column
df_raw['date'] = pd.to_datetime(df_raw['date'])
df_raw = df_raw.set_index('date').sort_index()

# Extract HTT columns (htt_A01 through htt_A15)
htt_cols = [col for col in df_raw.columns if col.startswith(f'{product}_A') and '/' not in col]
htt_cols = sorted(htt_cols, key=lambda x: int(x.split('_A')[1]))

print(f"Found HTT columns: {htt_cols}")
print(f"Date range: {df_raw.index.min()} to {df_raw.index.max()}")
print(f"Total observations: {len(df_raw)}")

# Extract price data
df_prices = df_raw[htt_cols].copy()

# Compute daily changes (Δprice_t, not % returns)
df_returns = df_prices.diff().dropna()

# Drop any rows with NA after differencing
df_returns = df_returns.dropna()

# Filter out holiday dates
initial_count = len(df_returns)
# Convert index dates to a list and check membership
is_holiday = [date.date() in holiday_dates for date in df_returns.index]
df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
removed_count = initial_count - len(df_returns)

print(f"\nReturns data shape: {df_returns.shape}")
print(f"Removed {removed_count} holiday dates from returns data (out of {initial_count} total)")
print(f"First few returns:")
print(df_returns.head())

Found HTT columns: ['clbr_A01', 'clbr_A02', 'clbr_A03', 'clbr_A04', 'clbr_A05', 'clbr_A06', 'clbr_A07', 'clbr_A08', 'clbr_A09', 'clbr_A10', 'clbr_A11', 'clbr_A12', 'clbr_A13', 'clbr_A14', 'clbr_A15']
Date range: 2022-01-03 00:00:00 to 2025-12-25 00:00:00
Total observations: 1041


: 

In [ ]:
df_returns.tail(50)

,clbr_A01,clbr_A02,clbr_A03,clbr_A04,clbr_A05,clbr_A06,clbr_A07,clbr_A08,clbr_A09,clbr_A10,clbr_A11,clbr_A12,clbr_A13,clbr_A14,clbr_A15
date,,,,,,,,,,,,,,,
2025-10-17,-7.000000e-02,-0.04,-0.03,-4.000000e-02,-3.000000e-02,-3.000000e-02,-3.000000e-02,-7.000000e-02,-6.000000e-02,-7.000000e-02,-7.000000e-02,-7.000000e-02,-7.000000e-02,-7.000000e-02,-7.000000e-02
2025-10-20,1.500000e-01,0.09,0.05,6.000000e-02,5.000000e-02,5.000000e-02,4.000000e-02,4.000000e-02,2.000000e-02,3.000000e-02,3.000000e-02,4.000000e-02,3.000000e-02,3.000000e-02,3.000000e-02
2025-10-21,-9.000000e-02,0.02,0.05,5.000000e-02,7.000000e-02,7.000000e-02,8.000000e-02,1.000000e-01,1.000000e-01,1.000000e-01,9.000000e-02,8.000000e-02,8.000000e-02,8.000000e-02,8.000000e-02
2025-10-22,1.700000e-01,0.06,0.05,3.000000e-02,0.000000e+00,1.000000e-02,0.000000e+00,-2.000000e-02,-2.000000e-02,-1.000000e-02,-7.993606e-15,-2.000000e-02,-1.000000e-02,-1.000000e-02,-1.000000e-02
2025-10-23,-2.200000e-01,-0.22,-0.22,-2.300000e-01,-2.100000e-01,-2.000000e-01,-1.800000e-01,-1.800000e-01,-1.800000e-01,-1.800000e-01,-1.700000e-01,-1.600000e-01,-1.600000e-01,-1.600000e-01,-1.600000e-01
2025-10-24,-1.100000e-01,-0.03,0.02,5.000000e-02,5.000000e-02,4.000000e-02,4.000000e-02,3.000000e-02,4.000000e-02,4.000000e-02,2.000000e-02,4.000000e-02,4.000000e-02,4.000000e-02,4.000000e-02
2025-10-27,1.200000e-01,0.12,0.12,1.100000e-01,9.000000e-02,9.000000e-02,7.000000e-02,6.000000e-02,5.000000e-02,5.000000e-02,6.000000e-02,3.000000e-02,3.000000e-02,3.000000e-02,3.000000e-02
2025-10-28,7.000000e-02,0.04,-0.01,-2.000000e-02,-1.000000e-02,-1.000000e-02,1.000000e-02,2.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02
2025-10-29,-1.100000e-01,-0.05,-0.02,-2.000000e-02,-1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,-6.661338e-15,0.000000e+00,6.661338e-15,-1.000000e-02,-1.000000e-02,-1.000000e-02,-1.000000e-02


## 3. Build Position Vector (w)

In [36]:
# Build node position vector from contract codes
# Initialize all nodes to zero
all_nodes = [f"A{i:02d}" for i in range(1, 16)]
w_node = pd.Series(0.0, index=all_nodes, name='lots')

# Map contract positions to nodes
for contract, lots in pos_codes.items():
    if contract in contract_to_node:
        node = contract_to_node[contract]
        w_node[node] = lots
        print(f"  {contract} ({node}): {lots} lots")

print(f"\nNode position vector:")
print(w_node)

# Create bucket position vectors
w_front = w_node[front].values
w_mid = w_node[mid].values
w_back = w_node[back].values
w_total = w_node.values

print(f"\nBucket positions:")
print(f"  Front: {w_front}")
print(f"  Mid: {w_mid}")
print(f"  Back: {w_back}")

  J6 (A02): -268.0004 lots
  N6 (A05): 300.0 lots
  U6 (A07): 850.0 lots
  V6 (A08): -850.0 lots
  Z6 (A10): 100.0 lots

Node position vector:
A01      0.0000
A02   -268.0004
A03      0.0000
A04      0.0000
A05    300.0000
A06      0.0000
A07    850.0000
A08   -850.0000
A09      0.0000
A10    100.0000
A11      0.0000
A12      0.0000
A13      0.0000
A14      0.0000
A15      0.0000
Name: lots, dtype: float64

Bucket positions:
  Front: [   0.     -268.0004    0.        0.    ]
  Mid: [ 300.    0.  850. -850.]
  Back: [  0. 100.   0.   0.   0.   0.   0.]


## 4. EWMA Covariance Engine

In [37]:
def compute_ewma_covariance(returns_df, nodes, lambda_val, init_obs=60):
    """
    Compute EWMA covariance matrix for given nodes.
    
    Parameters:
    -----------
    returns_df : DataFrame
        DataFrame with returns (daily changes)
    nodes : list
        List of node names (e.g., ['A01', 'A02', ...])
    lambda_val : float
        EWMA decay parameter
    init_obs : int
        Number of observations to use for initial covariance
    
    Returns:
    --------
    cov_matrix : ndarray
        Final EWMA covariance matrix
    """
    # Extract relevant columns
    cols = [f'{product}_{node}' for node in nodes]
    returns_subset = returns_df[cols].values
    
    n_obs, n_nodes = returns_subset.shape
    
    if n_obs < init_obs:
        raise ValueError(f"Need at least {init_obs} observations, got {n_obs}")
    
    # Initialize with sample covariance of first N observations
    init_returns = returns_subset[:init_obs]
    # Remove any rows with NaN
    init_returns = init_returns[~np.isnan(init_returns).any(axis=1)]
    
    if len(init_returns) < 10:
        # Fallback: use identity matrix scaled by variance
        cov_current = np.eye(n_nodes) * np.var(returns_subset, axis=0).mean()
    else:
        cov_current = np.cov(init_returns.T)
    
    # EWMA recursion: Σ_t = λ * Σ_{t-1} + (1-λ) * r_t * r_t'
    for t in range(init_obs, n_obs):
        r_t = returns_subset[t:t+1, :]  # Shape: (1, n_nodes)
        
        # Skip if any NaN
        if np.isnan(r_t).any():
            continue
        
        # EWMA update
        outer_product = np.outer(r_t, r_t)
        cov_current = lambda_val * cov_current + (1 - lambda_val) * outer_product[0]
    
    return cov_current

# Compute EWMA covariance for each bucket
print("Computing EWMA covariances...")

Sigma_front = compute_ewma_covariance(df_returns, front, lambda_front, ewma_init_obs)
Sigma_mid = compute_ewma_covariance(df_returns, mid, lambda_mid, ewma_init_obs)
Sigma_back = compute_ewma_covariance(df_returns, back, lambda_back, ewma_init_obs)

print(f"\nFront covariance shape: {Sigma_front.shape}")
print(f"Mid covariance shape: {Sigma_mid.shape}")
print(f"Back covariance shape: {Sigma_back.shape}")

# Build total covariance matrix (block-diagonal for now, Option 1)
# TODO: Option 2 - add cross-bucket correlations using lambda_cross
n_front = len(front)
n_mid = len(mid)
n_back = len(back)
n_total = n_front + n_mid + n_back

Sigma_total = np.zeros((n_total, n_total))
Sigma_total[:n_front, :n_front] = Sigma_front
Sigma_total[n_front:n_front+n_mid, n_front:n_front+n_mid] = Sigma_mid
Sigma_total[n_front+n_mid:, n_front+n_mid:] = Sigma_back

print(f"\nTotal covariance shape: {Sigma_total.shape}")

Computing EWMA covariances...

Front covariance shape: (4, 4)
Mid covariance shape: (4, 4)
Back covariance shape: (7, 7)

Total covariance shape: (15, 15)


## 5. Bucket Summary DataFrame

In [39]:
def compute_q_risk(w, Sigma):
    """Compute Q risk: Q = 1000 * sqrt(w' Σ w)"""
    var = w.T @ Sigma @ w
    if var < 0:
        return 0.0
    return 1000 * np.sqrt(var)

def compute_mc_to_total(w_bucket, w_total, Sigma_total, bucket_start_idx, bucket_nodes):
    """
    Compute marginal contribution of bucket to total portfolio.
    MC_bucket = 1000 * (w_bucket' Σ_total w_total) / sqrt(w_total' Σ_total w_total)
    """
    # Map bucket positions to total vector
    w_bucket_in_total = np.zeros(len(w_total))
    w_bucket_in_total[bucket_start_idx:bucket_start_idx+len(bucket_nodes)] = w_bucket
    
    # Compute numerator: w_bucket' Σ_total w_total
    numerator = w_bucket_in_total.T @ Sigma_total @ w_total
    
    # Compute denominator: sqrt(w_total' Σ_total w_total)
    total_var = w_total.T @ Sigma_total @ w_total
    if total_var <= 0:
        return 0.0
    denominator = np.sqrt(total_var)
    
    return 1000 * numerator / denominator

# Compute standalone Q for each bucket
Q_front = compute_q_risk(w_front, Sigma_front)
Q_mid = compute_q_risk(w_mid, Sigma_mid)
Q_back = compute_q_risk(w_back, Sigma_back)
Q_total = compute_q_risk(w_total, Sigma_total)

# Compute MC to total
MC_front = compute_mc_to_total(w_front, w_total, Sigma_total, 0, front)
MC_mid = compute_mc_to_total(w_mid, w_total, Sigma_total, n_front, mid)
MC_back = compute_mc_to_total(w_back, w_total, Sigma_total, n_front + n_mid, back)

# Build bucket_summary_df
bucket_summary_df = pd.DataFrame({
    'bucket': ['Front', 'Mid', 'Back', 'TOTAL'],
    'standalone_Q': [Q_front, Q_mid, Q_back, Q_total],
    'MC_to_total': [MC_front, MC_mid, MC_back, np.nan],
})

# Sanity checks
bucket_summary_df['Q_check'] = bucket_summary_df['standalone_Q'].apply(lambda x: 'OK' if x >= 0 and np.isfinite(x) else 'FAIL')

# MC_signed indicates the sign of the marginal contribution to total portfolio risk:
# - POS: Positive MC - this bucket increases total portfolio risk (diversification benefit < standalone risk)
# - NEG: Negative MC - this bucket decreases total portfolio risk (diversification benefit > standalone risk, i.e., hedging effect)
# - ZERO: Zero MC - this bucket has no net contribution to total risk (perfectly hedged or no positions)
bucket_summary_df['MC_signed'] = bucket_summary_df['MC_to_total'].apply(lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO')

print("Bucket Summary:")
print(bucket_summary_df.to_string(index=False))
print("\nNote: MC_signed shows whether the bucket contributes positively (POS) or negatively (NEG) to total portfolio risk.")
print("      A negative MC indicates the bucket provides diversification/hedging benefits to the overall portfolio.")

# Assertions
assert Q_total >= 0, "Total Q must be non-negative"
assert np.isfinite(Q_total), "Total Q must be finite"
assert all(bucket_summary_df['standalone_Q'] >= 0), "All bucket Qs must be non-negative"
print("\n✓ Sanity checks passed")

Bucket Summary:
bucket  standalone_Q  MC_to_total Q_check MC_signed
 Front  2.861847e+06 2.091604e+06      OK       POS
   Mid  2.601440e+06 1.728280e+06      OK       POS
  Back  6.126386e+05 9.585071e+04      OK       POS
 TOTAL  3.915735e+06          NaN      OK      ZERO

Note: MC_signed shows whether the bucket contributes positively (POS) or negatively (NEG) to total portfolio risk.
      A negative MC indicates the bucket provides diversification/hedging benefits to the overall portfolio.

✓ Sanity checks passed


In [22]:
bucket_summary_df

,bucket,standalone_Q,MC_to_total,Q_check,MC_signed
0,Front,157848.809798,130206.517630,OK,POS
1,Mid,107519.532370,60412.225439,OK,POS
2,Back,11905.461918,740.700437,OK,POS
3,TOTAL,191359.443506,NaN,OK,ZERO


## 6. Factor Detail DataFrame

In [18]:
def build_factor_matrix_bucket(nodes, bucket_type='front'):
    """
    Build factor matrix B for a bucket.
    
    For Front/Mid: Level + adjacent spreads (A01/A02, A02/A03, etc)
    For Back: Level + longer spreads + residual
    """
    n_nodes = len(nodes)
    
    if bucket_type in ['front', 'mid']:
        # Level factor: equal weight
        level_factor = np.ones(n_nodes) / n_nodes
        
        # Adjacent spreads
        spread_factors = []
        for i in range(n_nodes - 1):
            spread = np.zeros(n_nodes)
            spread[i] = 1
            spread[i+1] = -1
            spread_factors.append(spread)
        
        # Combine: Level + spreads
        B = np.column_stack([level_factor] + spread_factors)
        factor_names = ['Level'] + [f"{nodes[i]}/{nodes[i+1]}" for i in range(n_nodes-1)]
        
        return B, factor_names, None  # No residual for front/mid
    
    else:  # back bucket
        # Level factor: equal weight
        level_factor = np.ones(n_nodes) / n_nodes
        
        # Longer/coarse spreads
        # Block1: A09-A11, Block2: A12-A13, Block3: A14-A15
        # Block averages
        block1_avg = np.zeros(n_nodes)
        block1_avg[:3] = 1.0/3  # A09, A10, A11
        
        block2_avg = np.zeros(n_nodes)
        block2_avg[3:5] = 1.0/2  # A12, A13
        
        block3_avg = np.zeros(n_nodes)
        block3_avg[5:] = 1.0/2  # A14, A15
        
        # Coarse spreads
        spread1 = block1_avg - block2_avg  # Block1 - Block2
        spread2 = block2_avg - block3_avg  # Block2 - Block3
        
        # Early-back vs late-back
        early_back = np.zeros(n_nodes)
        early_back[:4] = 1.0/4  # A09-A12
        late_back = np.zeros(n_nodes)
        late_back[4:] = 1.0/3  # A13-A15
        spread3 = early_back - late_back
        
        # Additional spread: A09/A12 (skip some nodes)
        spread4 = np.zeros(n_nodes)
        spread4[0] = 1  # A09
        spread4[3] = -1  # A12
        
        # Combine factors
        B_coarse = np.column_stack([level_factor, spread1, spread2, spread3, spread4])
        factor_names_coarse = ['Level', 'Block1-Block2', 'Block2-Block3', 'Early-Late', 'A09/A12']
        
        return B_coarse, factor_names_coarse, 'residual'

def compute_factor_risk_metrics(w_bucket, Sigma_bucket, B, factor_names, Sigma_total, w_total, bucket_start_idx, n_total, residual_name=None):
    """
    Compute factor-level risk metrics using FULL covariance matrix.
    
    Factor decomposition: w = B * e, where:
    - w: node positions (n_nodes x 1)
    - B: factor matrix (n_nodes x n_factors), columns are factor loadings
    - e: factor exposures (n_factors x 1)
    
    For Level factor: B[:, 0] = [1/n, 1/n, ..., 1/n] (equal weight)
    Level exposure e[0] represents the total net position across all nodes.
    
    This function uses Sigma_total (full covariance) for risk calculations, so factor MCs
    show contribution to TOTAL portfolio risk and sum to bucket MC_to_total.
    
    Parameters:
    -----------
    w_bucket : ndarray
        Bucket node positions
    Sigma_bucket : ndarray
        Bucket covariance matrix (used only for factor exposure calculation)
    B : ndarray
        Bucket factor matrix
    factor_names : list
        Factor names
    Sigma_total : ndarray
        Full covariance matrix (n_total x n_total)
    w_total : ndarray
        Full portfolio positions (n_total x 1)
    bucket_start_idx : int
        Starting index of bucket nodes in full space
    n_total : int
        Total number of nodes
    residual_name : str, optional
        Name for residual factor (for back bucket)
    
    Returns:
    --------
    factor_df : DataFrame with columns: factor_name, qty_lots, marginal_slope, MC_$per_day, pct_of_bucket_Q, AS_skew_direction
    bucket_MC_to_total : float
        Bucket MC to total (for tie-out verification)
    recon_error : float
        Reconstruction error
    """
    # Factor exposures: e such that w = B * e
    # For square invertible B: e = B^{-1} w
    # For non-square or non-invertible: e = (B^T B)^{-1} B^T w (least squares)
    if B.shape[0] == B.shape[1]:
        # Square matrix - try direct inverse
        try:
            B_inv = np.linalg.inv(B)
            e = B_inv @ w_bucket
        except np.linalg.LinAlgError:
            # Not invertible, use pseudoinverse
            e = np.linalg.pinv(B) @ w_bucket
    else:
        # Non-square, use least squares
        e = np.linalg.lstsq(B, w_bucket, rcond=None)[0]
    
    # Verify: Level exposure should equal sum of node positions
    # (since Level = [1/n, 1/n, ..., 1/n], Level exposure = sum(w) / n * n = sum(w))
    level_idx = factor_names.index('Level') if 'Level' in factor_names else None
    if level_idx is not None:
        level_exposure = e[level_idx]
        total_net_position = w_bucket.sum()
        # Mathematical check: For Level = [1/n, 1/n, ..., 1/n], if w = B * e, then:
        # sum(w) = sum(B[:, 0]) * e_level + sum(B[:, 1]) * e_spread1 + ... 
        # Since sum(B[:, i]) = 0 for all spreads (they're differences), and sum(B[:, 0]) = n * (1/n) = 1,
        # we get: sum(w) = 1 * e_level, so e_level = sum(w)
        # This means Level exposure equals total net position, which is correct.
        level_check = abs(level_exposure - total_net_position)
        if level_check > 1e-6:
            print(f"WARNING: Level exposure ({level_exposure:.2f}) != total net position ({total_net_position:.2f}), diff={level_check:.6f}")
    
    # Map bucket factor matrix B to full space: B_full (n_total x n_factors)
    n_bucket_nodes = B.shape[0]
    n_factors = B.shape[1]
    B_full = np.zeros((n_total, n_factors))
    B_full[bucket_start_idx:bucket_start_idx+n_bucket_nodes, :] = B
    
    # Map bucket positions to full space for residual calculation
    w_bucket_in_total = np.zeros(n_total)
    w_bucket_in_total[bucket_start_idx:bucket_start_idx+n_bucket_nodes] = w_bucket
    
    # Factor covariance using FULL covariance matrix: Σ_f = B_full' Σ_total B_full
    Sigma_f = B_full.T @ Sigma_total @ B_full
    
    # Compute total portfolio variance for MC normalization
    total_var = w_total.T @ Sigma_total @ w_total
    sqrt_total_var = np.sqrt(total_var) if total_var > 0 else 1e-10
    
    # Compute bucket MC to total for tie-out verification
    # MC_bucket = 1000 * (w_bucket_in_total' Σ_total w_total) / sqrt(w_total' Σ_total w_total)
    bucket_mc_numerator = w_bucket_in_total.T @ Sigma_total @ w_total
    bucket_MC_to_total = 1000 * bucket_mc_numerator / sqrt_total_var if sqrt_total_var > 1e-10 else 0.0
    
    # Marginal risk slopes using full covariance: slope_f = B_full' Σ_total w_total
    # This gives the marginal contribution of each factor to total portfolio risk
    slope_f = B_full.T @ Sigma_total @ w_total
    
    # Factor MCs using Euler allocation: MC_factor = 1000 * (e * slope_f) / sqrt(w_total' Σ_total w_total)
    # This ensures sum(MC_factor) = bucket_MC_to_total
    # Proof: sum(MC_factor) = 1000 * sum(e * slope_f) / sqrt_total_var
    #                      = 1000 * (e' slope_f) / sqrt_total_var
    #                      = 1000 * (e' B_full' Σ_total w_total) / sqrt_total_var
    #                      = 1000 * (w_bucket_in_total' Σ_total w_total) / sqrt_total_var
    #                      = bucket_MC_to_total
    MC_factor = 1000 * (e * slope_f) / sqrt_total_var if sqrt_total_var > 1e-10 else np.zeros(len(e))
    
    # Verify reconstruction: w_recon = B * e should equal w_bucket (within numerical tolerance)
    w_recon = B @ e
    recon_error = np.linalg.norm(w_recon - w_bucket)
    
    # Percentage of bucket MC_to_total (not standalone Q)
    if abs(bucket_MC_to_total) > 1e-10:
        pct_of_bucket_MC = 100 * MC_factor / bucket_MC_to_total
    else:
        pct_of_bucket_MC = np.zeros(len(e))
    
    # AS skew direction
    AS_direction = pd.Series(slope_f).apply(lambda x: 'SELL' if x > 0 else 'BUY' if x < 0 else 'NEUTRAL')
    
    # Build DataFrame
    factor_df = pd.DataFrame({
        'factor_name': factor_names,
        'qty_lots': e,
        'marginal_slope': slope_f,
        'MC_$per_day': MC_factor,
        'pct_of_bucket_Q': pct_of_bucket_MC,  # Now percentage of bucket MC_to_total
        'AS_skew_direction': AS_direction
    })
    
    # Handle residual if needed
    if residual_name:
        # Compute residual component using full covariance
        # Project bucket positions onto factor space: w_exp = B * e (already computed as w_recon)
        # But we need to compute the projection using full covariance for MC calculation
        # P_bucket = B (B' Σ_bucket B)^+ B' Σ_bucket (for exposure calculation)
        # For MC, we need: residual_MC = MC(w_bucket) - MC(w_exp) where w_exp is in factor space
        
        # Compute projected bucket positions in factor space
        w_exp_bucket = B @ e  # This is w_recon, the projection onto factor space
        
        # Residual bucket positions (what's not explained by factors)
        w_res_bucket = w_bucket - w_exp_bucket
        
        # Map residual to full space for MC calculation
        w_res_full = np.zeros(n_total)
        w_res_full[bucket_start_idx:bucket_start_idx+n_bucket_nodes] = w_res_bucket
        
        # Residual risk contribution (Euler) using full covariance
        if np.linalg.norm(w_res_bucket) > 1e-10:
            # Residual MC = 1000 * (w_res_full' Σ_total w_total) / sqrt(w_total' Σ_total w_total)
            residual_MC = 1000 * (w_res_full.T @ Sigma_total @ w_total) / sqrt_total_var if sqrt_total_var > 1e-10 else 0.0
            # Residual slope: marginal contribution of residual to total portfolio risk
            residual_slope = (w_res_full.T @ Sigma_total @ w_total) / np.linalg.norm(w_res_bucket) if np.linalg.norm(w_res_bucket) > 1e-10 else 0
            
            residual_row = pd.DataFrame({
                'factor_name': [residual_name],
                'qty_lots': [np.linalg.norm(w_res_bucket)],
                'marginal_slope': [residual_slope],
                'MC_$per_day': [residual_MC],
                'pct_of_bucket_Q': [100 * residual_MC / bucket_MC_to_total if abs(bucket_MC_to_total) > 1e-10 else 0],
                'AS_skew_direction': ['NEUTRAL']
            })
            
            factor_df = pd.concat([factor_df, residual_row], ignore_index=True)
    
    return factor_df, bucket_MC_to_total, recon_error

# Build factor detail for each bucket
print("Building factor detail DataFrames...")

# Front bucket
B_front, factor_names_front, _ = build_factor_matrix_bucket(front, 'front')
print(f"\nFront bucket positions: {w_front}")
print(f"Front bucket total net position: {w_front.sum()}")
print(f"Front Level factor vector (B[:, 0]): {B_front[:, 0]}")
print(f"\nMathematical check:")
print(f"  If Level exposure = e_level, then Level contributes (1/4)*e_level to each node")
print(f"  Level contribution per node = (1/4) * e_level")
print(f"  Sum of all node positions = 4 * (1/4) * e_level = e_level")
print(f"  Therefore: e_level = sum(positions) = {w_front.sum()}")
factor_detail_front, MC_front_check, recon_error_front = compute_factor_risk_metrics(
    w_front, Sigma_front, B_front, factor_names_front, 
    Sigma_total, w_total, 0, n_total
)
factor_detail_front.insert(0, 'bucket', 'Front')
level_exposure = factor_detail_front[factor_detail_front['factor_name'] == 'Level']['qty_lots'].values[0]
print(f"\nFront Level qty_lots (factor exposure): {level_exposure}")
print(f"  Level contribution to each node: (1/4) * {level_exposure} = {0.25 * level_exposure}")
print(f"  But this is just the Level component - spreads adjust from this base")
print(f"Front: MC_to_total={MC_front_check:.2f}, recon_error={recon_error_front:.2e}")

# Mid bucket
B_mid, factor_names_mid, _ = build_factor_matrix_bucket(mid, 'mid')
factor_detail_mid, MC_mid_check, recon_error_mid = compute_factor_risk_metrics(
    w_mid, Sigma_mid, B_mid, factor_names_mid,
    Sigma_total, w_total, n_front, n_total
)
factor_detail_mid.insert(0, 'bucket', 'Mid')
print(f"Mid: MC_to_total={MC_mid_check:.2f}, recon_error={recon_error_mid:.2e}")

# Back bucket
B_back, factor_names_back, residual_name = build_factor_matrix_bucket(back, 'back')
factor_detail_back, MC_back_check, recon_error_back = compute_factor_risk_metrics(
    w_back, Sigma_back, B_back, factor_names_back,
    Sigma_total, w_total, n_front + n_mid, n_total, residual_name
)
factor_detail_back.insert(0, 'bucket', 'Back')
print(f"Back: MC_to_total={MC_back_check:.2f}, recon_error={recon_error_back:.2e}")

# Combine all buckets
factor_detail_df = pd.concat([factor_detail_front, factor_detail_mid, factor_detail_back], ignore_index=True)

print("\nFactor Detail DataFrame:")
print(factor_detail_df.to_string(index=False))

# Explanation about factor MCs using full covariance
print("\n" + "="*80)
print("FACTOR MCs USE FULL COVARIANCE MATRIX")
print("="*80)
print("Factor MCs in factor_detail_df are computed using the FULL covariance matrix (Σ_total):")
front_level_mc = factor_detail_df[(factor_detail_df['bucket'] == 'Front') & (factor_detail_df['factor_name'] == 'Level')]['MC_$per_day'].values[0]
mid_level_mc = factor_detail_df[(factor_detail_df['bucket'] == 'Mid') & (factor_detail_df['factor_name'] == 'Level')]['MC_$per_day'].values[0]
front_mc_to_total = bucket_summary_df[bucket_summary_df['bucket'] == 'Front']['MC_to_total'].values[0]
mid_mc_to_total = bucket_summary_df[bucket_summary_df['bucket'] == 'Mid']['MC_to_total'].values[0]
total_q = bucket_summary_df[bucket_summary_df['bucket'] == 'TOTAL']['standalone_Q'].values[0]

print(f"  Front Level MC (contribution to total portfolio): {front_level_mc:.2f}")
print(f"  Mid Level MC (contribution to total portfolio): {mid_level_mc:.2f}")
print(f"  Sum of Level MCs: {front_level_mc + mid_level_mc:.2f}")
print(f"\n  Front bucket MC_to_total: {front_mc_to_total:.2f}")
print(f"  Mid bucket MC_to_total: {mid_mc_to_total:.2f}")
print(f"  Sum of bucket MCs: {front_mc_to_total + mid_mc_to_total:.2f}")
print(f"  Total Q: {total_q:.2f}")
print(f"\n  Key insight:")
print("  - Factor MCs show contribution to TOTAL portfolio risk (using Σ_total)")
print("  - Factor MCs within each bucket sum to that bucket's MC_to_total")
print("  - This accounts for cross-bucket correlations and diversification effects")
print("  - Sum of all factor MCs across all buckets = Total Q")
print("="*80)

print("\n" + "="*80)
print("FACTOR DECOMPOSITION EXPLANATION:")
print("="*80)
print("For Front/Mid buckets, factors are: Level + adjacent spreads")
print("\nLevel Factor:")
print("  - Level factor vector: [1/n, 1/n, ..., 1/n] = [0.25, 0.25, 0.25, 0.25] for Front")
print("  - Level exposure (qty_lots) = TOTAL NET POSITION across all nodes")
print("  - Mathematical relationship:")
print("    * Level contributes (1/n) * e_level to each node")
print("    * Sum of all positions = n * (1/n) * e_level = e_level")
print("    * Therefore: Level exposure = sum(positions)")
print("\n  Example:")
print(f"    * Node positions: {w_front}")
print(f"    * Total net position: {w_front.sum()}")
print(f"    * Level exposure: {w_front.sum()} (NOT the average!)")
print(f"    * Level contributes {0.25 * w_front.sum():.0f} to each node (before spreads)")
print("\nSpread Factors:")
print("  - Each spread (e.g., A01/A02) represents the difference between adjacent nodes")
print("  - Spread exposures adjust positions from the Level base")
print("  - Final position = Level contribution + spread adjustments")
print("\n  Example reconstruction:")
print(f"    * A01 final = {w_front[0]}")
print(f"    * A01 = Level_base + spread_adjustments")
print(f"    * Level_base = (1/4) * {w_front.sum()} = {0.25 * w_front.sum():.0f}")
print(f"    * Spread adjustments = {w_front[0] - 0.25 * w_front.sum():.0f}")
print("="*80)

# Tie-out assertions - factor MCs should sum to bucket MC_to_total
for bucket_name, MC_check, MC_actual in [('Front', MC_front_check, MC_front), 
                                         ('Mid', MC_mid_check, MC_mid), 
                                         ('Back', MC_back_check, MC_back)]:
    if abs(MC_actual) > 1e-10:
        rel_error = abs(MC_check - MC_actual) / abs(MC_actual)
        assert rel_error < 1e-3, f"{bucket_name} MC_to_total tie-out failed: {rel_error:.6f}"
        print(f"\n✓ {bucket_name} MC_to_total tie-out (computed vs bucket_summary): {MC_check:.2f} vs {MC_actual:.2f} (error: {rel_error:.6f})")
    elif abs(MC_check) < 1e-6 and abs(MC_actual) < 1e-6:
        print(f"\n✓ {bucket_name} MC_to_total tie-out: both zero (no positions)")

# Check factor MC sums - should equal bucket MC_to_total
for bucket_name, MC_target in [('Front', MC_front_check), ('Mid', MC_mid_check), ('Back', MC_back_check)]:
    bucket_factors = factor_detail_df[factor_detail_df['bucket'] == bucket_name]
    mc_sum = bucket_factors['MC_$per_day'].sum()
    if abs(MC_target) > 1e-10:  # Use small threshold to avoid division by zero
        rel_error = abs(mc_sum - MC_target) / abs(MC_target)
        assert rel_error < 1e-3, f"{bucket_name} factor MC sum tie-out failed: {rel_error:.6f}"
        print(f"✓ {bucket_name} factor MC sum tie-out: {mc_sum:.2f} vs {MC_target:.2f} (error: {rel_error:.6f})")
    elif abs(mc_sum) < 1e-6 and abs(MC_target) < 1e-6:
        # Both are effectively zero - this is correct
        print(f"✓ {bucket_name} factor MC sum tie-out: both zero (no positions)")
    else:
        # One is zero but the other isn't - this is an error
        abs_error = abs(mc_sum - MC_target)
        assert abs_error < 1e-6, f"{bucket_name} factor MC sum tie-out failed: MC_sum={mc_sum:.2f}, MC_target={MC_target:.2f}, abs_error={abs_error:.6f}"
        print(f"✓ {bucket_name} factor MC sum tie-out: {mc_sum:.2f} vs {MC_target:.2f} (both effectively zero)")

Building factor detail DataFrames...

Front bucket positions: [  100. -1400. -1974. -1400.]
Front bucket total net position: -4674.0
Front Level factor vector (B[:, 0]): [0.25 0.25 0.25 0.25]

Mathematical check:
  If Level exposure = e_level, then Level contributes (1/4)*e_level to each node
  Level contribution per node = (1/4) * e_level
  Sum of all node positions = 4 * (1/4) * e_level = e_level
  Therefore: e_level = sum(positions) = -4674.0

Front Level qty_lots (factor exposure): -4674.0
  Level contribution to each node: (1/4) * -4674.0 = -1168.5
  But this is just the Level component - spreads adjust from this base
Front: MC_to_total=130206.52, recon_error=0.00e+00
Mid: MC_to_total=60412.23, recon_error=0.00e+00
Back: MC_to_total=740.70, recon_error=1.94e+02

Factor Detail DataFrame:
bucket   factor_name      qty_lots  marginal_slope   MC_$per_day  pct_of_bucket_Q AS_skew_direction
 Front         Level -4.674000e+03   -5.330819e+00  1.302065e+05     1.000000e+02               B

## 7. Level vs Structure DataFrame

In [19]:
# Aggregate factor_detail_df into Level vs Structure per bucket
# Note: Factor MCs now use full covariance, so they sum to bucket MC_to_total (not standalone Q)
level_structure_rows = []

for bucket_name in ['Front', 'Mid', 'Back']:
    bucket_factors = factor_detail_df[factor_detail_df['bucket'] == bucket_name].copy()
    
    # Level contribution
    level_row = bucket_factors[bucket_factors['factor_name'] == 'Level'].iloc[0]
    level_MC = level_row['MC_$per_day']
    
    # Structure contribution = sum of all spreads + residual
    structure_factors = bucket_factors[bucket_factors['factor_name'] != 'Level']
    structure_MC = structure_factors['MC_$per_day'].sum()
    
    # Bucket MC_to_total (should equal Level + Structure)
    bucket_MC = bucket_summary_df[bucket_summary_df['bucket'] == bucket_name]['MC_to_total'].values[0]
    
    level_structure_rows.append({
        'bucket': bucket_name,
        'Level': level_MC,
        'Structure': structure_MC,
        'MC_to_total': bucket_MC,
        'Level_pct': 100 * level_MC / bucket_MC if abs(bucket_MC) > 1e-10 else 0,
        'Structure_pct': 100 * structure_MC / bucket_MC if abs(bucket_MC) > 1e-10 else 0
    })

level_structure_df = pd.DataFrame(level_structure_rows)

print("Level vs Structure DataFrame:")
print(level_structure_df.to_string(index=False))
print("\nNote: Level and Structure MCs sum to bucket MC_to_total (contribution to total portfolio risk)")

# Verify Level + Structure = MC_to_total
for _, row in level_structure_df.iterrows():
    total_check = row['Level'] + row['Structure']
    mc_target = row['MC_to_total']
    if abs(mc_target) > 1e-10:
        rel_error = abs(total_check - mc_target) / abs(mc_target)
        assert rel_error < 1e-3, f"{row['bucket']} Level+Structure tie-out failed: {rel_error:.6f}"
        print(f"\n✓ {row['bucket']} Level+Structure tie-out: {total_check:.2f} vs {mc_target:.2f} (error: {rel_error:.6f})")
    elif abs(total_check) < 1e-6 and abs(mc_target) < 1e-6:
        print(f"\n✓ {row['bucket']} Level+Structure tie-out: both zero")

Level vs Structure DataFrame:
bucket         Level    Structure   MC_to_total  Level_pct  Structure_pct
 Front 130206.517630 2.689180e-09 130206.517630 100.000000   2.065319e-12
   Mid  60412.225362 7.739476e-05  60412.225439 100.000000   1.281111e-07
  Back    740.634939 6.549797e-02    740.700437  99.991157   8.842708e-03

Note: Level and Structure MCs sum to bucket MC_to_total (contribution to total portfolio risk)

✓ Front Level+Structure tie-out: 130206.52 vs 130206.52 (error: 0.000000)

✓ Mid Level+Structure tie-out: 60412.23 vs 60412.23 (error: 0.000000)

✓ Back Level+Structure tie-out: 740.70 vs 740.70 (error: 0.000000)


## 8. Top Drivers + Skew Direction

In [20]:
# Extract top factor by absolute MC for each bucket
# Note: MC_$per_day values come from factor_detail_df, which uses FULL covariance matrix (Σ_total)
# These are marginal contributions to TOTAL portfolio risk, not standalone bucket Q
top_drivers_rows = []

for bucket_name in ['Front', 'Mid', 'Back']:
    bucket_factors = factor_detail_df[factor_detail_df['bucket'] == bucket_name].copy()
    
    # Find factor with maximum absolute MC (marginal contribution to total portfolio)
    # MC_$per_day is computed using: MC = 1000 * (exposure * slope) / sqrt(w_total' Σ_total w_total)
    # where slope = B_full' @ Σ_total @ w_total (marginal contribution to total portfolio risk)
    bucket_factors['abs_MC'] = bucket_factors['MC_$per_day'].abs()
    top_factor = bucket_factors.loc[bucket_factors['abs_MC'].idxmax()]
    
    top_drivers_rows.append({
        'bucket': bucket_name,
        'top_factor_by_abs_MC': top_factor['factor_name'],
        'factor_qty': top_factor['qty_lots'],
        'factor_slope': top_factor['marginal_slope'],  # Slope using full covariance: B_full' @ Σ_total @ w_total
        'factor_MC': top_factor['MC_$per_day'],  # MC to total portfolio using full covariance
        'AS_direction': top_factor['AS_skew_direction']
    })

top_drivers_df = pd.DataFrame(top_drivers_rows)

print("Top Drivers + Skew Direction:")
print("(All values use FULL covariance matrix - MC shows contribution to TOTAL portfolio risk)")
print(top_drivers_df.to_string(index=False))

Top Drivers + Skew Direction:
(All values use FULL covariance matrix - MC shows contribution to TOTAL portfolio risk)
bucket top_factor_by_abs_MC  factor_qty  factor_slope     factor_MC AS_direction
 Front                Level     -4674.0     -5.330819 130206.517630          BUY
   Mid                Level      4600.0      2.513141  60412.225362         SELL
  Back                Level      -675.0     -0.209967    740.634939          BUY


## 9. Hedge Recommender

In [ ]:
def determine_bucket_for_instrument(instrument_name, front_nodes, mid_nodes, back_nodes):
    """
    Determine which bucket an instrument (spread or outright) belongs to.
    
    Parameters:
    -----------
    instrument_name : str
        Instrument name, either a spread (e.g., "A03/A04") or an outright (e.g., "A01")
    
    Returns:
    --------
    tuple: (bucket_type, bucket_nodes) or None if cannot determine
    """
    # Check if it's a spread (contains '/')
    if '/' in instrument_name:
        # Parse spread (e.g., "A03/A04")
        parts = instrument_name.split('/')
        if len(parts) != 2:
            return None
        
        node1, node2 = parts[0], parts[1]
        
        # Check if both nodes are in the same bucket
        if node1 in front_nodes and node2 in front_nodes:
            return 'front', front_nodes
        elif node1 in mid_nodes and node2 in mid_nodes:
            return 'mid', mid_nodes
        elif node1 in back_nodes and node2 in back_nodes:
            return 'back', back_nodes
        
        # If spread spans buckets, determine primary bucket
        if node1 in front_nodes or node2 in front_nodes:
            return 'front', front_nodes
        elif node1 in mid_nodes or node2 in mid_nodes:
            return 'mid', mid_nodes
        else:
            return 'back', back_nodes
    else:
        # It's an outright (e.g., "A01")
        node = instrument_name
        
        # Determine which bucket the node belongs to
        if node in front_nodes:
            return 'front', front_nodes
        elif node in mid_nodes:
            return 'mid', mid_nodes
        elif node in back_nodes:
            return 'back', back_nodes
        else:
            return None

def build_hedge_universe(bucket_nodes, bucket_type, returns_df, product):
    """Build hedge universe for a bucket."""
    hedge_instruments = []
    
    # Outrights in bucket
    for node in bucket_nodes:
        hedge_instruments.append(node)
    
    # Adjacent spreads
    for i in range(len(bucket_nodes) - 1):
        spread_name = f"{bucket_nodes[i]}/{bucket_nodes[i+1]}"
        hedge_instruments.append(spread_name)
    
    # For back bucket, add longer/coarse spreads
    if bucket_type == 'back':
        # Add some longer spreads
        if len(bucket_nodes) >= 4:
            hedge_instruments.append(f"{bucket_nodes[0]}/{bucket_nodes[3]}")  # A09/A12
        if len(bucket_nodes) >= 6:
            hedge_instruments.append(f"{bucket_nodes[2]}/{bucket_nodes[5]}")  # A11/A14
    
    return hedge_instruments

def compute_instrument_returns(returns_df, instrument_name, bucket_nodes, product):
    """Compute returns for an instrument (outright or spread)."""
    if '/' not in instrument_name:
        # Outright
        col = f"{product}_{instrument_name}"
        if col in returns_df.columns:
            return returns_df[col].values
        else:
            return None
    else:
        # Spread
        parts = instrument_name.split('/')
        node1, node2 = parts[0], parts[1]
        col1 = f"{product}_{node1}"
        col2 = f"{product}_{node2}"
        
        if col1 in returns_df.columns and col2 in returns_df.columns:
            return returns_df[col1].values - returns_df[col2].values
        else:
            return None

def compute_ewma_variance(returns_series, lambda_val, init_obs=60):
    """Compute EWMA variance for a single series."""
    returns_array = returns_series[~np.isnan(returns_series)]
    n_obs = len(returns_array)
    
    if n_obs < init_obs:
        return np.var(returns_array) if n_obs > 1 else 0.0
    
    # Initialize with sample variance
    var_current = np.var(returns_array[:init_obs])
    
    # EWMA recursion
    for t in range(init_obs, n_obs):
        r_t = returns_array[t]
        if not np.isnan(r_t):
            var_current = lambda_val * var_current + (1 - lambda_val) * r_t**2
    
    return var_current

def compute_ewma_covariance_pair(returns1, returns2, lambda_val, init_obs=60):
    """Compute EWMA covariance between two series."""
    # Align series
    valid_mask = ~(np.isnan(returns1) | np.isnan(returns2))
    r1 = returns1[valid_mask]
    r2 = returns2[valid_mask]
    
    n_obs = len(r1)
    
    if n_obs < init_obs:
        return np.cov(r1[:init_obs], r2[:init_obs])[0, 1] if n_obs > 1 else 0.0
    
    # Initialize with sample covariance
    cov_current = np.cov(r1[:init_obs], r2[:init_obs])[0, 1]
    
    # EWMA recursion
    for t in range(init_obs, n_obs):
        r1_t = r1[t]
        r2_t = r2[t]
        if not (np.isnan(r1_t) or np.isnan(r2_t)):
            cov_current = lambda_val * cov_current + (1 - lambda_val) * r1_t * r2_t
    
    return cov_current

def recommend_hedge(target_instrument, q_target_lots, product, returns_df, 
                    front_nodes, mid_nodes, back_nodes,
                    lambda_front, lambda_mid, lambda_back):
    """
    Recommend hedge for a target instrument (spread or outright).
    
    Parameters:
    -----------
    target_instrument : str
        Target instrument, either a spread (e.g., "A03/A04") or an outright (e.g., "A01")
    q_target_lots : float
        Target position size in lots
    
    Returns:
    --------
    hedge_summary_df : DataFrame sorted by var_reduction desc
    """
    # Determine bucket
    bucket_info = determine_bucket_for_instrument(target_instrument, front_nodes, mid_nodes, back_nodes)
    if bucket_info is None:
        raise ValueError(f"Cannot determine bucket for instrument {target_instrument}")
    
    bucket_type, bucket_nodes = bucket_info
    
    # Get lambda for this bucket
    lambda_map = {'front': lambda_front, 'mid': lambda_mid, 'back': lambda_back}
    lambda_val = lambda_map[bucket_type]
    
    # Build hedge universe
    hedge_universe = build_hedge_universe(bucket_nodes, bucket_type, returns_df, product)
    
    # Compute target returns
    target_returns = compute_instrument_returns(returns_df, target_instrument, bucket_nodes, product)
    if target_returns is None:
        raise ValueError(f"Cannot compute returns for target {target_instrument}")
    
    # Compute EWMA covariance for target
    target_var = compute_ewma_variance(target_returns, lambda_val, ewma_init_obs)
    
    # Evaluate each hedge instrument
    hedge_results = []
    
    for hedge_instrument in hedge_universe:
        hedge_returns = compute_instrument_returns(returns_df, hedge_instrument, bucket_nodes, product)
        if hedge_returns is None:
            continue
        
        # Compute covariance between target and hedge
        cov_target_hedge = compute_ewma_covariance_pair(target_returns, hedge_returns, lambda_val, ewma_init_obs)
        hedge_var = compute_ewma_variance(hedge_returns, lambda_val, ewma_init_obs)
        
        if hedge_var <= 0:
            continue
        
        # Hedge ratio: beta = Cov(T,H) / Var(H)
        beta = cov_target_hedge / hedge_var
        
        # Variance reduction: 1 - Var(T - beta*H) / Var(T)
        if target_var > 0:
            hedged_var = target_var - 2 * beta * cov_target_hedge + beta**2 * hedge_var
            var_reduction = 1 - hedged_var / target_var
        else:
            var_reduction = 0.0
        
        # Correlation: Corr = Cov / (sqrt(VarT * VarH))
        if target_var > 0 and hedge_var > 0:
            corr = cov_target_hedge / np.sqrt(target_var * hedge_var)
        else:
            corr = 0.0
        
        # Recommended hedge lots: x = -q_target_lots * beta
        recommended_hedge_lots = -q_target_lots * beta
        
        hedge_results.append({
            'hedge_instrument': hedge_instrument,
            'corr': corr,
            'beta': beta,
            'var_reduction': var_reduction,
            'recommended_hedge_lots_for_q_target': recommended_hedge_lots
        })
    
    hedge_summary_df = pd.DataFrame(hedge_results)
    hedge_summary_df = hedge_summary_df.sort_values('var_reduction', ascending=False)
    
    return hedge_summary_df

# Example: Recommend hedge for a spread or outright
print("Hedge Recommender Example:")
print("Example 1: Target spread A03/A04, q_target_lots = 1000")
print("\n" + "="*80)

hedge_summary_spread = recommend_hedge(
    target_instrument="A03/A04",
    q_target_lots=1000,
    product=product,
    returns_df=df_returns,
    front_nodes=front,
    mid_nodes=mid,
    back_nodes=back,
    lambda_front=lambda_front,
    lambda_mid=lambda_mid,
    lambda_back=lambda_back
)

print("\nHedge Summary for Spread A03/A04 (sorted by variance reduction):")
print(hedge_summary_spread.to_string(index=False))

print("\n\nExample 2: Target outright A01, q_target_lots = 1000")
print("="*80)

hedge_summary_outright = recommend_hedge(
    target_instrument="A01",
    q_target_lots=1000,
    product=product,
    returns_df=df_returns,
    front_nodes=front,
    mid_nodes=mid,
    back_nodes=back,
    lambda_front=lambda_front,
    lambda_mid=lambda_mid,
    lambda_back=lambda_back
)

print("\nHedge Summary for Outright A01 (sorted by variance reduction):")
print(hedge_summary_outright.to_string(index=False))

Hedge Recommender Example:
Example 1: Target spread A03/A04, q_target_lots = 1000


Hedge Summary for Spread A03/A04 (sorted by variance reduction):
hedge_instrument      corr      beta  var_reduction  recommended_hedge_lots_for_q_target
         A03/A04  1.000000  1.000000       1.000000                         -1000.000000
             A03  0.512566  0.393944       0.262724                          -393.943531
             A02  0.468688  0.283847       0.219669                          -283.847468
             A01  0.368025  0.197833       0.135443                          -197.833325
             A04 -0.285721 -0.245086       0.081637                           245.086455
         A02/A03  0.105432  0.103896       0.011116                          -103.896059
         A01/A02 -0.074044 -0.061417       0.005483                            61.417226


Example 2: Target outright A01, q_target_lots = 1000

Hedge Summary for Outright A01 (sorted by variance reduction):
hedge_instrument    

## 10. Final Summary

In [44]:
print("="*80)
print("Q RISK REPORT SUMMARY")
print("="*80)

print("\n1. BUCKET SUMMARY:")
print(bucket_summary_df.to_string(index=False))

print("\n\n2. LEVEL VS STRUCTURE:")
print(level_structure_df.to_string(index=False))

print("\n\n3. TOP DRIVERS:")
print(top_drivers_df.to_string(index=False))

print("\n\n4. FACTOR DETAIL (first 20 rows):")
print(factor_detail_df.head(20).to_string(index=False))

print("\n\n" + "="*80)
print("Report generation complete.")
print("="*80)

Q RISK REPORT SUMMARY

1. BUCKET SUMMARY:
bucket  standalone_Q   MC_to_total Q_check MC_signed
 Front 146968.676584 130302.608258      OK       POS
   Mid  76672.690085  35463.773442      OK       POS
  Back      0.000000      0.000000      OK      ZERO
 TOTAL 165766.381700           NaN      OK      ZERO


2. LEVEL VS STRUCTURE:
bucket         Level    Structure   MC_to_total  Level_pct  Structure_pct
 Front 130302.608258 5.964779e-08 130302.608258      100.0   4.577636e-11
   Mid  35463.773346 9.638538e-05  35463.773442      100.0   2.717855e-07
  Back      0.000000 1.657664e+05      0.000000        0.0   0.000000e+00


3. TOP DRIVERS:
bucket top_factor_by_abs_MC  factor_qty  factor_slope     factor_MC AS_direction
 Front                Level      3000.0      7.199931 130302.608258         SELL
   Mid                Level     -3000.0     -1.959567  35463.773346          BUY
  Back                Level         0.0      0.000000      0.000000      NEUTRAL


4. FACTOR DETAIL (first 20 r